In [5]:
import requests
import yaml
from time import sleep
from pprint import pprint

############
# Gathers all terms and borrowed Ontologies employed by Chemistry ontologies, listed in ontologies_details.yml
# var chem_ontos_terms - is dictionary holding ontology terms organized by ontology id/prefix
# var all_ontos - is dictionary where holding info on ontologies employed by chemestry ontologies, organized by ontology id/prefix
############



def yaml2dict(path):
    with open(path, 'r') as yaml_f:
        yaml_content = yaml_f.read()
        yaml_dict = yaml.safe_load(yaml_content)
    return yaml_dict

def get_all_ontology_terms(api_url, size, page, terms):
    '''
    return dict with all terms of an ontology, 
    each entry with keys: iri, id, label, onto_prefix
    '''
    api_url_w_params = f'{api_url}?page={page}&size={size}'
    # print(api_url_w_params)
    try:
        request = requests.get(api_url_w_params)
    except requests.exceptions.Timeout:
        sleep(1)
        request = requests.get(api_url_w_params)
    response = request.json()
    page_numb = response['page']['number']
    page_total  = response['page']['totalPages']
    # print(f'Current page:{page_numb}  Total pages:{page_total}')
    if '_embedded' in response.keys():
        terms.update({term['obo_id']: {'iri': term['iri'], 'id': term['obo_id'], 'label': term['label'], 'onto_prefix': term['obo_id'].split(':')[0]} for term in response['_embedded']['terms'] if term['obo_id']})
        get_all_ontology_terms(api_url=api_url, size=size, page=page+1, terms=terms)
    return terms

def get_onto_info(onto_prefix):
    onto_api_url =  f'{tib_ols_api}ontologies/{onto_prefix}'
    onto_request = requests.get(onto_api_url)
    if onto_request.status_code == 404:
        onto_api_url =  f'{ebispot_ols_api}ontologies/{onto_prefix}'
        onto_request = requests.get(onto_api_url)
        if onto_request.status_code == 404:
            # print(f'No ontology info in OLS for {onto_prefix}')
            onto_response = {}     
        else:
            # print(f'getting {onto_prefix} from EBISPOT OLS')
            onto_response = onto_request.json() 
    else:
        # print(f'getting {onto_prefix} from TIB OLS')
        onto_response = onto_request.json() 
    return onto_response


tib_ols_api = 'https://service.tib.eu/ts4tib/api/'
ebispot_ols_api = 'http://www.ebi.ac.uk/ols/api/'
all_ontos = {} 
chem_ontos = yaml2dict(path='ontologies_details.yml')
chem_ontos_ids = [onto['id'].upper() for onto in chem_ontos]
chem_ontos_terms = {}
# for onto in ontos_used_in_ontos:
#     print(onto)
#     onto_response = get_onto_info(onto_prefix=onto)
#     pprint(onto_response)

for onto in chem_ontos:
    print(f"Ontology: {onto['id']}")
    all_onto_terms = get_all_ontology_terms(api_url=f'{tib_ols_api}ontologies/{onto["id"]}/terms', size=100, page=0, terms={})
    print(f"Ontology: {onto['id']} has {len(all_onto_terms)} terms")
    chem_ontos_terms[onto['id'].upper()] = all_onto_terms
    # pprint(all_onto_terms)

    ontos_used_in_ontos = set([v['onto_prefix'] for v in all_onto_terms.values()])
    print(f"Ontology: {onto['id']} uses the {', '.join(ontos_used_in_ontos)} ontologies")

    new_ontos_used_by_onto = {onto: get_onto_info(onto_prefix=onto) for onto in ontos_used_in_ontos if onto not in all_ontos.keys()}  # get info on ontology
    all_ontos.update(new_ontos_used_by_onto)  # & append to var all_ontos
    print(f"All ontologies: {all_ontos.keys()}")
    print(f'\n{"*"*20}\n')
    # pprint(all_ontos['PR'])

# pprint(chem_ontos_terms) # all terms for all chem ontologies listed in ontologies_details.yml
print(chem_ontos_terms.keys())

# enter missing chem_ontos instances into all_ontos
for chem_onto in chem_ontos_ids: 
    if chem_onto not in all_ontos.keys():
        print(chem_onto, "missing")
        new_entry = {chem_onto: {}}
        all_ontos.update(new_entry)


Ontology: RXNO
Ontology: RXNO has 901 terms
Ontology: RXNO uses the CHEBI, IAO, OBI, RXNO, BFO, MOP ontologies
All ontologies: dict_keys(['CHEBI', 'IAO', 'OBI', 'RXNO', 'BFO', 'MOP'])

********************

Ontology: MOP
Ontology: MOP has 3682 terms
Ontology: MOP uses the REX, CHEBI, RXNO, BFO, MOP ontologies
All ontologies: dict_keys(['CHEBI', 'IAO', 'OBI', 'RXNO', 'BFO', 'MOP', 'REX'])

********************

Ontology: CHEMINF
Ontology: CHEMINF has 832 terms
Ontology: CHEMINF uses the CHEMINF, CHEBI, CDK, IAO, PR, OBI, PATO, BFO, UO, GO, APOLLO_SV, GAZ ontologies
All ontologies: dict_keys(['CHEBI', 'IAO', 'OBI', 'RXNO', 'BFO', 'MOP', 'REX', 'CHEMINF', 'CDK', 'PR', 'PATO', 'UO', 'GO', 'APOLLO_SV', 'GAZ'])

********************

Ontology: CHMO
Ontology: CHMO has 3084 terms
Ontology: CHMO uses the CHMO, REX, CHEBI, IAO, OBI, FIX, BFO, MS, OBCS ontologies
All ontologies: dict_keys(['CHEBI', 'IAO', 'OBI', 'RXNO', 'BFO', 'MOP', 'REX', 'CHEMINF', 'CDK', 'PR', 'PATO', 'UO', 'GO', 'APOLLO_SV',

In [ ]:
# pprint(all_ontos['CDK'])
# print(all_ontos['CDK'].keys())
pprint(all_ontos['RXNO']['config'])
# all_ontos['CDK'].keys() is True

# chem_ontos_ids = [onto['id'].upper() for onto in chem_ontos]
# print(chem_ontos_ids)

# pprint(chem_ontos_terms['rxno'])
print('onto_dict', onto_dict.keys())
print(all_ontos.keys())
pprint(all_ontos['CDK'])
pprint(all_ontos['CHIRO'])
list(chem_ontos_terms.keys())[:1]

In [16]:
from pyvis.network import Network



target_onto = 'CHEMINF'

# for chem_onto in list(chem_ontos_terms.keys())[:1]:
opts = '''
    var options = {
    "interaction": {
        "navigationButtons": true
    },
    "physics": {
        "repulsion": {
        "centralGravity": 2.9,
        "springLength": 140,
        "nodeDistance": 255,
        "damping": 0.08
        },
        "maxVelocity": 3,
        "minVelocity": 0.13,
        "solver": "repulsion",
        "timestep": 0.48
    },
    "nodes": {
        "font": {
        "size": 16
        }
    },
    "edges": {
        "arrows": {
        "to": {
            "enabled": true,
            "scaleFactor": 0
        }
    }
  }
}
'''


def draw_onto_relations(target_onto):
    net = Network(directed=True, bgcolor='#0A0A0A', notebook=True)  
    net.width = '1000px'
    net.height = '750px'
    net.add_node(target_onto, label=target_onto, color="#6BFFF5", borderWidth=0, shape="circle", size=1500)
    for i, (onto_term_k, dest_onto_term) in enumerate(chem_ontos_terms[target_onto].items()):
        if i:
            # add ontologies nodes
            onto =  dest_onto_term['onto_prefix']
            if onto in chem_ontos_ids:
                color='#C46BFF'
            else:
                color='#ffff00'
            net.add_node(onto, label=onto,  color=color, borderWidth=0, shape="circle", size=500)
            if dest_onto_term['onto_prefix'] != target_onto:
                net.add_edge(target_onto, dest_onto_term['onto_prefix'], color='#ACACAC', title=f"{dest_onto_term['id']} - {dest_onto_term['label']} ")
    net.set_options(opts)
    # net.show(f'{target_onto}.html')
    net.save_graph(f'{target_onto}.html')



In [17]:
draw_onto_relations(target_onto='RXNO')

In [18]:
draw_onto_relations(target_onto='CHIRO')

In [ ]:
# unuse functions

def get_term_info_from_src_onto(iri):
    term_page =  f'{tib_ols_api}terms/findByIdAndIsDefiningOntology?iri={iri}'
    try:
        termrequest = requests.get(term_page)
    except requests.exceptions.Timeout:
        sleep(1)
        termrequest = requests.get(term_page)

    termresponse = termrequest.json()
    if termresponse['page']['totalElements'] == 0:
        # QUERY ANOTHER N ANOTHER OLS INSTANCE if ontology not in TIB OLS
        term_page =  f'{ebispot_ols_api}terms/findByIdAndIsDefiningOntology?iri={iri}'
        termrequest = requests.get(term_page)
        termresponse = termrequest.json()    
    print(term_page)
    # pprint(termresponse)
    if '_embedded' in termresponse.keys():
        term = termresponse['_embedded']['terms'][0]
        term_info = {
            # 'api': term_api_url,
            'term_iri': iri,
            # 'label': apiresponse['label'],
            # '_id': apiresponse['annotation']['id'][0] if 'id' in apiresponse['annotation'] else None,
            # 'obo_id': apiresponse['obo_id']['id'] if type(apiresponse['obo_id']) is dict and 'id' in apiresponse['obo_id'].keys() else apiresponse['obo_id'],
            'onto_iri': term['ontology_iri'],
            'onto_prefix': term['ontology_prefix'],
            'onto_name': term['ontology_name'],
            }
        return term_info
    else:
        return None



def terms_hierarchy(api_url):
    # calls OLS API on ontology terms 
    # and requests the terms children, recursively 
    apirequest = requests.get(api_url)
    apiresponse = apirequest.json() 
    if '_embedded' in apiresponse.keys():
        terms = apiresponse['_embedded']['terms']
        for term in terms:
            # pprint(term)
            term_iri = term['iri']
            print(f'\nterm_iri: {term_iri}')
            # ols_term_url = term['_links']['self']['href']
            # print(ols_term_url)
            term_info =get_term_info_from_src_onto(iri=term_iri)

            pprint(term_info)
            # if 'children' in term['_links'].keys():
            #     api_children_url = term['_links']['children']['href']
            #     terms_hierarchy(api_url=term['_links']['children']['href'])
            